In [2]:
!pip install flask pyngrok torch sentence-transformers==2.6.1 transformers==4.40.0 pillow -q


In [3]:
from pyngrok import ngrok

# Get your free token from https://ngrok.com
ngrok.set_auth_token("36FTWFbgPcDWfBIThIToJAYJaq8_5mMLqMeAd1bTQFHWEdtTQ")

In [4]:
# from flask import Flask, request, jsonify
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import base64
# from io import BytesIO
# from PIL import Image
# import tempfile
# import os

# app = Flask(__name__)

# print("=" * 60)
# print("🔄 Loading CheXagent-2-3b model on GPU...")
# print("=" * 60)

# model_name = "StanfordAIMI/CheXagent-2-3b"
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Device: {device}")

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     trust_remote_code=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     trust_remote_code=True,
#     torch_dtype=torch.float16  # Keep as float16
# )

# model.eval()

# print("=" * 60)
# print("✅ Model successfully loaded on GPU!")
# print(f"✅ Model: {model_name}")
# print(f"✅ Device: {device}")
# print("=" * 60)

# @app.route('/', methods=['GET'])
# def health_check():
#     """Health check endpoint"""
#     return jsonify({
#         'status': 'online',
#         'message': 'CheXagent API is running',
#         'device': str(device),
#         'model': model_name,
#         'gpu_available': torch.cuda.is_available(),
#         'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'
#     })

# @app.route('/analyze', methods=['POST'])
# def analyze_xray():
#     """Analyze chest X-ray image"""
#     try:
#         data = request.json

#         if not data or 'image' not in data:
#             return jsonify({
#                 'status': 'error',
#                 'error': 'No image data provided'
#             }), 400

#         print("📥 Receiving image data...")
#         image_data = base64.b64decode(data['image'])

#         with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
#             tmp.write(image_data)
#             tmp_path = tmp.name

#         print(f"💾 Image saved temporarily at: {tmp_path}")
#         print("🔬 Generating radiology report...")

#         prompt = "Generate a radiology report for this chest X-ray."

#         query = tokenizer.from_list_format([
#             {'image': tmp_path},
#             {'text': prompt}
#         ])

#         conv = [
#             {"from": "system", "value": "You are a helpful assistant."},
#             {"from": "human", "value": query}
#         ]

#         input_ids = tokenizer.apply_chat_template(
#             conv,
#             add_generation_prompt=True,
#             return_tensors="pt"
#         )

#         print("🚀 Running model inference...")

#         # FIX 1: Ensure input_ids are in the correct dtype (float16)
#         input_ids = input_ids.to(device)

#         # FIX 2: Use torch.no_grad() and ensure consistent dtype
#         with torch.no_grad():
#           # Removed autocast completely! ✅
#           output = model.generate(
#               input_ids,
#               do_sample=False,
#               num_beams=1,
#               max_new_tokens=512
#           )[0]

#         report = tokenizer.decode(output[input_ids.size(1):-1])

#         print("✅ Report generated successfully!")

#         os.unlink(tmp_path)
#         print("🗑️ Temporary file cleaned up")

#         return jsonify({
#             'status': 'success',
#             'report': report
#         })

#     except Exception as e:
#         print(f"❌ Error during analysis: {str(e)}")
#         import traceback
#         traceback.print_exc()  # Print full traceback for debugging
#         return jsonify({
#             'status': 'error',
#             'error': str(e)
#         }), 500

from flask import Flask, request, jsonify
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import base64
from io import BytesIO
from PIL import Image
import tempfile
import os

app = Flask(__name__)

print("=" * 60)
print("🔄 Loading CheXagent-2-3b model on GPU...")
print("=" * 60)

model_name = "StanfordAIMI/CheXagent-2-3b"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float32  # Use float32 for ALL components
)

# Ensure the entire model is in float32
model = model.float()

model.eval()

print("=" * 60)
print("✅ Model successfully loaded on GPU!")
print(f"✅ Model: {model_name}")
print(f"✅ Device: {device}")
print("=" * 60)

@app.route('/', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'online',
        'message': 'CheXagent API is running',
        'device': str(device),
        'model': model_name,
        'gpu_available': torch.cuda.is_available(),
        'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'
    })

@app.route('/analyze', methods=['POST'])
def analyze_xray():
    """Analyze chest X-ray image"""
    try:
        data = request.json

        if not data or 'image' not in data:
            return jsonify({
                'status': 'error',
                'error': 'No image data provided'
            }), 400

        print("📥 Receiving image data...")
        image_data = base64.b64decode(data['image'])

        with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
            tmp.write(image_data)
            tmp_path = tmp.name

        print(f"💾 Image saved temporarily at: {tmp_path}")
        print("🔬 Generating radiology report...")

        prompt = "Generate a radiology report for this chest X-ray."

        query = tokenizer.from_list_format([
            {'image': tmp_path},
            {'text': prompt}
        ])

        conv = [
            {"from": "system", "value": "You are a helpful assistant."},
            {"from": "human", "value": query}
        ]

        input_ids = tokenizer.apply_chat_template(
            conv,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        print("🚀 Running model inference...")

        # Move input to device
        input_ids = input_ids.to(device)

        # CRITICAL FIX: Disable autocast - model handles dtypes internally
        with torch.no_grad():
            output = model.generate(
                input_ids,
                do_sample=False,
                num_beams=1,
                max_new_tokens=512
            )[0]

        report = tokenizer.decode(output[input_ids.size(1):-1])

        print("✅ Report generated successfully!")

        os.unlink(tmp_path)
        print("🗑️ Temporary file cleaned up")

        return jsonify({
            'status': 'success',
            'report': report
        })

    except Exception as e:
        print(f"❌ Error during analysis: {str(e)}")
        import traceback
        traceback.print_exc()  # Print full traceback for debugging
        return jsonify({
            'status': 'error',
            'error': str(e)
        }), 500

🔄 Loading CheXagent-2-3b model on GPU...
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: Fut

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model successfully loaded on GPU!
✅ Model: StanfordAIMI/CheXagent-2-3b
✅ Device: cuda


In [5]:
print(model.dtype)  # Should show float16
print(next(model.parameters()).dtype)  # Check actual parameter dtype

torch.float32
torch.float32


In [6]:
import threading

# Create ngrok tunnel
print("=" * 60)
print("🌐 Creating ngrok tunnel...")
print("=" * 60)

public_url = ngrok.connect(5000)

print("=" * 60)
print(f"✅ PUBLIC URL: {public_url}")
print("=" * 60)
print("")
print("📋 COPY THIS URL AND PASTE IT IN YOUR STREAMLIT APP!")
print("")
print("Example: https://xxxx-xx-xxx-xxx-xxx.ngrok-free.app/analyze")
print("=" * 60)
print("")

# Start Flask server in background thread
def run_flask():
    """Run Flask server"""
    app.run(port=5000, use_reloader=False)

# Create and start thread
thread = threading.Thread(target=run_flask, daemon=True)
thread.start()

print("✅ Flask server is running!")
print("✅ Keep this Colab tab open while using the app")
print("=" * 60)
print("")
print("📊 Server Status:")
print(f"   • Model: {model_name}")
print(f"   • Device: {device}")
print(f"   • GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f"   • Public URL: {public_url}")
print("")
print("=" * 60)
print("🎉 Setup complete! Ready to receive requests.")
print("=" * 60)

# Keep the script running
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n👋 Shutting down gracefully...")

🌐 Creating ngrok tunnel...
✅ PUBLIC URL: NgrokTunnel: "https://retroserrulate-zackary-ogreishly.ngrok-free.dev" -> "http://localhost:5000"

📋 COPY THIS URL AND PASTE IT IN YOUR STREAMLIT APP!

Example: https://xxxx-xx-xxx-xxx-xxx.ngrok-free.app/analyze

✅ Flask server is running!
✅ Keep this Colab tab open while using the app

📊 Server Status:
   • Model: StanfordAIMI/CheXagent-2-3b
   • Device: cuda
   • GPU: Tesla T4
   • Public URL: NgrokTunnel: "https://retroserrulate-zackary-ogreishly.ngrok-free.dev" -> "http://localhost:5000"

🎉 Setup complete! Ready to receive requests.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 15:07:29] "GET / HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📥 Receiving image data...
💾 Image saved temporarily at: /tmp/tmpkw55r5k2.png
🔬 Generating radiology report...
🚀 Running model inference...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 15:07:58] "POST /analyze HTTP/1.1" 200 -


✅ Report generated successfully!
🗑️ Temporary file cleaned up


...

✅ Report generated successfully!
🗑️ Temporary file cleaned up


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 15:24:23] "GET / HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📥 Receiving image data...
💾 Image saved temporarily at: /tmp/tmpsyzcltst.png
🔬 Generating radiology report...
🚀 Running model inference...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 15:24:45] "POST /analyze HTTP/1.1" 200 -


✅ Report generated successfully!
🗑️ Temporary file cleaned up

👋 Shutting down gracefully...
